In [31]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [32]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import sys

#sys.path.append('/content/gdrive/MyDrive/NNProject/')

train_ann_df = pd.read_csv('/content/drive/MyDrive/NNProject/train_data.csv')
super_map_df = pd.read_csv('/content/drive/MyDrive/NNProject/superclass_mapping.csv')
sub_map_df = pd.read_csv('/content/drive/MyDrive/NNProject/subclass_mapping.csv')

train_img_dir = '/content/drive/MyDrive/NNProject/train_shuffle'
test_img_dir = '/content/drive/MyDrive/NNProject/test_shuffle'

In [35]:
"""
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'
"""

image_preprocessing = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0), std=(1)),
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [36]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
                        nn.Conv2d(3, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2, 2)
                      )

        self.block2 = nn.Sequential(
                        nn.Conv2d(32, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2, 2)
                      )

        self.block3 = nn.Sequential(
                        nn.Conv2d(64, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2, 2)
                      )

        self.fc1 = nn.Linear(4*4*128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3a = nn.Linear(128, 4)
        self.fc3b = nn.Linear(128, 88)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        super_out = self.fc3a(x)
        sub_out = self.fc3b(x)
        return super_out, sub_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device

    def train_epoch(self):
        running_loss = 0.0
        #device = self.device
        for i, data in enumerate(self.train_loader):
            #inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)
            inputs, super_labels, sub_labels = data[0], data[1], data[3]

            self.optimizer.zero_grad()
            super_outputs, sub_outputs = self.model(inputs)
            loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss/i:.3f}')

    def validate_epoch(self):
        super_correct = 0
        sub_correct = 0
        total = 0
        running_loss = 0.0
        #device = self.device
        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                #inputs, super_labels, sub_labels = data[0].to(device), data[1].to(device), data[3].to(device)
                inputs, super_labels, sub_labels = data[0], data[1], data[3]

                super_outputs, sub_outputs = self.model(inputs)
                loss = self.criterion(super_outputs, super_labels) + self.criterion(sub_outputs, sub_labels)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                total += super_labels.size(0)
                super_correct += (super_predicted == super_labels).sum().item()
                sub_correct += (sub_predicted == sub_labels).sum().item()
                running_loss += loss.item()

        print(f'Validation loss: {running_loss/i:.3f}')
        print(f'Validation superclass acc: {100 * super_correct / total:.2f} %')
        print(f'Validation subclass acc: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        # Evaluate on test set, in this simple demo no special care is taken for novel/unseen classes
        test_predictions = {'image': [], 'superclass_index': [], 'subclass_index': []}
        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                #inputs, img_name = data[0].to(device), data[1]
                inputs, img_name = data[0], data[1]

                super_outputs, sub_outputs = self.model(inputs)
                _, super_predicted = torch.max(super_outputs.data, 1)
                _, sub_predicted = torch.max(sub_outputs.data, 1)

                test_predictions['image'].append(img_name[0])
                test_predictions['superclass_index'].append(super_predicted.item())
                test_predictions['subclass_index'].append(sub_predicted.item())

        test_predictions = pd.DataFrame(data=test_predictions)

        if save_to_csv:
            test_predictions.to_csv('example_test_predictions1.csv', index=False)

        if return_predictions:
            return test_predictions

In [7]:
# Init model and trainer
#device = 'cuda'
#model = CNN().to(device)
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

In [8]:
# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Training loss: 4.259
Validation loss: 3.941
Validation superclass acc: 86.23 %
Validation subclass acc: 15.98 %

Epoch 2
Training loss: 3.183
Validation loss: 3.319
Validation superclass acc: 87.82 %
Validation subclass acc: 27.06 %

Epoch 3
Training loss: 2.586
Validation loss: 2.744
Validation superclass acc: 89.72 %
Validation subclass acc: 35.76 %

Epoch 4
Training loss: 2.123
Validation loss: 2.529
Validation superclass acc: 92.09 %
Validation subclass acc: 43.67 %

Epoch 5
Training loss: 1.769
Validation loss: 2.396
Validation superclass acc: 91.77 %
Validation subclass acc: 47.15 %

Epoch 6
Training loss: 1.429
Validation loss: 2.242
Validation superclass acc: 93.51 %
Validation subclass acc: 50.47 %

Epoch 7
Training loss: 1.135
Validation loss: 2.497
Validation superclass acc: 90.51 %
Validation subclass acc: 50.16 %

Epoch 8
Training loss: 0.985
Validation loss: 2.339
Validation superclass acc: 92.41 %
Validation subclass acc: 50.63 %

Epoch 9
Training loss: 0.731
Val

In [9]:
trainer.test(save_to_csv=False, return_predictions=True)

'''
This simple baseline scores the following test accuracy

Superclass Accuracy
Overall: 43.83 %
Seen: 61.11 %
Unseen: 0.00 %

Subclass Accuracy
Overall: 2.03 %
Seen: 9.56 %
Unseen: 0.00 %
'''

'\nThis simple baseline scores the following test accuracy\n\nSuperclass Accuracy\nOverall: 43.83 %\nSeen: 61.11 %\nUnseen: 0.00 %\n\nSubclass Accuracy\nOverall: 2.03 %\nSeen: 9.56 %\nUnseen: 0.00 %\n'

In [16]:
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)
output = trainer.test(save_to_csv=False, return_predictions=True)
output

,image,superclass_index,subclass_index
0,0.jpg,0,30
1,1.jpg,0,46
2,2.jpg,2,69
3,3.jpg,0,20
4,4.jpg,1,24
...,...,...,...
12372,12372.jpg,0,30
12373,12373.jpg,2,72
12374,12374.jpg,2,71
12375,12375.jpg,1,7


In [37]:
import torch.nn as nn
from torch.nn import functional as F
#from torchvision.models import MobileNetV2
from torchvision.models import mobilenet_v2

class CustomMobileNetV2(nn.Module):
    def __init__(self, num_classes_super, num_classes_sub):
        super(CustomMobileNetV2, self).__init__()

        self.mobilenet = mobilenet_v2(pretrained=True)
        #self.mobilenet = MobileNetV2()

        # Adjust classifier for super-class classification
        self.fc1 = nn.Linear(1280, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3a = nn.Linear(128, num_classes_super)

        # Additional fully connected layer for sub-class classification
        self.fc3b = nn.Linear(128, num_classes_sub)

    def forward(self, x):
        features = self.mobilenet.features(x)

        x = F.adaptive_avg_pool2d(features, (1, 1))
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        super_out = self.fc3a(x)
        sub_out = self.fc3b(x)

        return super_out, sub_out


# Example usage:
num_classes_super = 4  # Adjust this based on the number of super-classes
num_classes_sub = 88   # Adjust this based on the number of sub-classes
model_mob = CustomMobileNetV2(num_classes_super, num_classes_sub)
#model_mod = CustomMobileNetV2()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mob.parameters(), lr=1e-3)
trainer = Trainer(model_mob, criterion, optimizer, train_loader, val_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [38]:
# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Training loss: 3.919
Validation loss: 3.550
Validation superclass acc: 90.35 %
Validation subclass acc: 22.47 %

Epoch 2
Training loss: 2.784
Validation loss: 3.080
Validation superclass acc: 91.77 %
Validation subclass acc: 27.69 %

Epoch 3
Training loss: 2.366
Validation loss: 2.710
Validation superclass acc: 93.20 %
Validation subclass acc: 35.92 %

Epoch 4
Training loss: 2.018
Validation loss: 2.431
Validation superclass acc: 95.25 %
Validation subclass acc: 39.72 %

Epoch 5
Training loss: 1.775
Validation loss: 2.407
Validation superclass acc: 94.62 %
Validation subclass acc: 40.03 %

Epoch 6
Training loss: 1.569
Validation loss: 2.335
Validation superclass acc: 95.09 %
Validation subclass acc: 45.25 %

Epoch 7
Training loss: 1.397
Validation loss: 2.230
Validation superclass acc: 94.46 %
Validation subclass acc: 48.10 %

Epoch 8
Training loss: 1.221
Validation loss: 2.212
Validation superclass acc: 94.62 %
Validation subclass acc: 51.11 %

Epoch 9
Training loss: 1.157
Val

In [39]:
output = trainer.test(save_to_csv=False, return_predictions=True)
print(output)

ValueError: ignored

## Scratch Codes

In [21]:
from torchvision.models import mobilenet_v2
"""
#import torch
model_mob = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mob.parameters(), lr=1e-3)
trainer = Trainer(model_mob, criterion, optimizer, train_loader, val_loader, test_loader)
"""

# Load MobileNetV2 from torchvision.models
model_mob = mobilenet_v2(pretrained=True)

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = 3  # Change this to the actual number of classes in your dataset
model_mob.classifier[1] = nn.Linear(model_mob.classifier[1].in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mob.parameters(), lr=1e-3)

# Assuming Trainer is a custom class for training, replace it with your actual training logic
trainer = Trainer(model_mob, criterion, optimizer, train_loader, val_loader, test_loader)

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import MobileNetV2
from torchvision.models.mobilenet import ConvBNReLU

class CustomMobileNetV2(nn.Module):
    def __init__(self, num_classes_super, num_classes_sub):
        super(CustomMobileNetV2, self).__init__()
        self.mobilenet = MobileNetV2()

        # Customize the classifier for super-class classification
        self.classifier_super = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.mobilenet.classifier[1].in_features, num_classes_super),
        )

        # Additional layers for sub-class classification
        self.conv1_sub = ConvBNReLU(1280, 256, kernel_size=1)
        self.avgpool_sub = nn.AdaptiveAvgPool2d(1)
        self.classifier_sub = nn.Linear(256, num_classes_sub)

    def forward(self, x):
        features = self.mobilenet.features(x)
        x = self.mobilenet.avgpool(features)
        x = x.view(x.size(0), -1)

        super_out = self.classifier_super(x)

        x = self.conv1_sub(features)
        x = self.avgpool_sub(x)
        x = x.view(x.size(0), -1)
        sub_out = self.classifier_sub(x)

        return super_out, sub_out

# Example usage:
num_classes_super = 3  # Adjust this based on the number of super-classes
num_classes_sub = 10   # Adjust this based on the number of sub-classes
model = CustomMobileNetV2(num_classes_super, num_classes_sub)